# Quality Measure Portal

## special nodes
 - Pass id = 99
 - Fail id = 100
 - Exclude = 101

**Google Doc** (https://docs.google.com/document/d/1hz8OYZcb7H7SLtQxQCfPwHDus3tHoxYjuuokjQJmE1k/edit#heading=h.ar22s947w3x)


patient ID = 647e416edb75e9b76ea46b3f

## Node types
-  **hasAValue**  : check if there is a value within the given path or not.
-  **inAList**    : check if the fetched item existed in the given list or not.
-  **boolean**    : the retrieved value is either True or False.
-  **YesNo**      : check if there is a Yes or No within the given path.
-  **lessThan**   : check if the fetched item is less than the given value.
-  **LargerThan** : check if the fetched item is larger than the given value.
-  **inRange**    : check if the fetched item existed within a list of two values.


In [106]:
# Functions

import json
from datetime import datetime
import re

def TNMStaging(text):
    # Patters to match T, N, and M followed by 0 or X
    t_pattern = re.compile(r'T[0Xx]')
    n_pattern = re.compile(r'N[0Xx]')
    m_pattern = re.compile(r'M[0Xx]')

    # check for matches
    t_match = t_pattern.search(text)
    n_match = n_pattern.search(text)
    m_match = m_pattern.search(text)

    # return true only if all matches are found, otherwise return false
    return bool(t_match and n_match and m_match)

def convert_time(T):
    timestamp = int(T) / 1000  # Convert from milliseconds to seconds
    dt_object = datetime.fromtimestamp(timestamp)
    formatted_date_time = dt_object.strftime('%Y-%m-%d %H:%M:%S')
    return formatted_date_time


def checkTime(t1, t2):
    t1 = int(t1)
    t2 = int(t2)
    
    t1 /=1000
    t2 /=1000
    
    dt1 = datetime.fromtimestamp(t1)
    dt2 = datetime.fromtimestamp(t2)
    
    print(f'QMTwoDates1: {dt1}')
    print(f'QMTwoDates2: {dt2}')
    
    if dt1==dt2:
        return True
    else:
        return False
    
# def TNMStaging(text):
#     t_pattern = re.compile(r'T\d+')
#     n_pattern = re.compile(r'N\d+')
#     m_pattern = re.compile(r'M\d+')

#     t_match = t_pattern.search(text)
#     n_match = n_pattern.search(text)
#     m_match = m_pattern.search(text)

#     t_value = t_match.group() if t_match else "Not found"
#     n_value = n_match.group() if n_match else "Not found"
#     m_value = m_match.group() if m_match else "Not found"

#     print("T component:", t_value)
#     print("N component:", n_value)
#     print("M component:", m_value)

TNMStaging("cT1cNxM0")


T component: T1
N component: Not found
M component: M0


In [98]:

class QMNode:

    # Read node from json and initialize required data
    # Include evaluation method, required paths, expected value(s)
    # and traversal info
    # Override with derived classes as needed.  Especially end states.

    def __init__(self, tree, nodeInfo):
        self.tree          = tree
        self.featurePath   = nodeInfo['path']
        self.successValues = nodeInfo['successValues']
        self.doEval        = nodeInfo['doEval']
        self.excludeValues = nodeInfo['excludeValues']
        self.YesBranchID   = nodeInfo['YesBranchID']
        self.NoBranchID    = nodeInfo['NoBranchID']
        # self.featureType   = nodeInfo['featureType']

    # Called by other nodes, or the tree, to evaluate the node and pass
    # execution to the next node
    
    def evaluate(self, patientData):
        fetchedValue = self.evalExpression(self.featurePath, patientData)
        result = False
        if self.featureType == "Boolean":
            result = fetchedValue
        
        
        # elif self.featureType == "YesNo":
        #     if fetchedValue in self.successValues:
        #         result = True
        #     else:
        #         result = False
        
        
        # elif self.featureType == "InAList":
        #     if fetchedValue in self.successValues:
        #         result = True
        #     else:
        #         result = False
        
        
        elif self.featureType == "LessThan":
            if fetchedValue < self.successValues:
                result = True
            else:
                result = False

        elif self.featureType == "LargerThan":
            if fetchedValue > self.successValues:
                result = True
            else:
                result = False

        elif self.featureType == "InARange":
            if fetchedValue > self.successValues[0] and fetchedValue <= self.successValues[1]:
                result = True
            else:
                result = False

        if result:
            return self.tree.getNode(self.YesBranchID).evaluate(patientData)
        
        else:
            return self.tree.getNode(self.NoBranchID).evaluate(patientData)

        
        
    def evalExpression(self, featurePath, patientData):
        print('\nEvaluating the following expression:\t', featurePath)
        path = featurePath.split('.')
        for feature in path:
            if feature in patientData:
                # print(feature,'\t', patientData)
                print(feature)
                patientData = patientData.get(feature)
                if type(patientData)== list:
                    sorted_data = sorted(patientData, key=lambda x: x["date"], reverse=True)
                    patientData = sorted_data[0]
            else:
                return None
        return patientData

In [99]:
class QMEndNode():
    def __init__(self, tree, nodeInfo):
        self.tree = tree
        self.featurePath = ""
        self.endState = nodeInfo['result']

    def evaluate(self, patientData=None):
        return str(self.endState)

In [100]:
class QMHasAValue(QMNode):
    def __init__(self, tree, nodeInfo=None):
        self.tree = tree
        self.featurePath   = nodeInfo['path']        
        self.successValues = nodeInfo['successValues']
        self.excludeValues = nodeInfo['excludeValues']
        self.YesBranchID   = nodeInfo['YesBranchID']
        self.NoBranchID    = nodeInfo['NoBranchID']
        # print(f'featurePath = {self.featurePath}')
        # print(f'successValues = {self.successValues}')


    def evaluate(self, patientData=None):
        
        fetchedValue = self.evalExpression(self.featurePath, patientData)
        print(f'QMHasAValue: {fetchedValue}')
        if fetchedValue and fetchedValue!="":
            return self.tree.getNode(self.YesBranchID).evaluate(patientData)        
        else:
            return self.tree.getNode(self.NoBranchID).evaluate(patientData)


In [101]:
class QMInAList(QMNode):
    def __init__(self, tree, nodeInfo=None):
        self.tree          = tree
        self.featurePath   = nodeInfo['path']        
        self.successValues = nodeInfo['successValues']
        self.excludeValues = nodeInfo['excludeValues']
        self.YesBranchID   = nodeInfo['YesBranchID']
        self.NoBranchID    = nodeInfo['NoBranchID']
        # print(f'featurePath = {self.featurePath}')
        # print(f'successValues = {self.successValues}')

    def evaluate(self, patientData=None):
        fetchedValue = self.evalExpression(self.featurePath, patientData)
        print(f'QMInAList: {fetchedValue}')

        excluding = False
        if len(self.excludeValues)>0:
            for item in self.excludeValues:
                if item in fetchedValue:
                    excluding = True
                    break

        if not excluding:
                if fetchedValue in self.successValues:
                    return self.tree.getNode(self.YesBranchID).evaluate(patientData)        
                else:
                    return self.tree.getNode(self.NoBranchID).evaluate(patientData)
        else:
            return self.tree.getNode(self.NoBranchID).evaluate(patientData)



In [102]:
class QMList(QMNode):
    def __init__(self, tree, nodeInfo=None):
        self.tree          = tree
        self.featurePath   = nodeInfo['path']        
        self.successValues = nodeInfo['successValues']
        self.YesBranchID   = nodeInfo['YesBranchID']
        self.NoBranchID    = nodeInfo['NoBranchID']
        # print(f'featurePath = {self.featurePath}')
        # print(f'successValues = {self.successValues}')


    def evaluate(self, patientData=None):
        
        fetchedValue = self.evalExpression(self.featurePath, patientData)
        print(f'QMList: {fetchedValue}')
        if fetchedValue and fetchedValue!="":
            if fetchedValue in self.successValues:
                return self.tree.getNode(self.YesBranchID).evaluate(patientData)        
            else:
                return self.tree.getNode(self.NoBranchID).evaluate(patientData)

In [103]:
class QMTwoDates(QMNode):
    '''In QMTwoDates, 
        successVaules feature holds two values of Date1 and Date2, 
        
        Date1<Date2

        This node return True if Date1<Date2 and False otherwise.
        '''
    def __init__(self, tree, nodeInfo=None):
        self.tree          = tree
        self.featurePath   = nodeInfo['path']        
        self.successValues = nodeInfo['successValues']
        self.YesBranchID   = nodeInfo['YesBranchID']
        self.NoBranchID    = nodeInfo['NoBranchID']
        # print(f'featurePath = {self.featurePath}')
        # print(f'successValues = {self.successValues}')


    def evaluate(self, patientData=None):        
        fetchedValue1 = self.evalExpression(self.successValues[0], patientData)
        fetchedValue2 = self.evalExpression(self.successValues[1], patientData)

        print(f'QMTwoDates1: {convert_time(fetchedValue1)}')
        print(f'QMTwoDates2: {convert_time(fetchedValue2)}')
        
        if  (fetchedValue1!= None) and (fetchedValue1!=None):
            return self.tree.getNode(self.YesBranchID).evaluate(patientData)        
        else:
            return self.tree.getNode(self.NoBranchID).evaluate(patientData)

In [104]:
class QMTree:
    dt_json = None
    def __init__(self):
        self.nodes = dict()

    def readTreeFile(self, path):
        with open(f'{path}.json', 'r') as f:
            self.tree = json.load(f)
            
            
        for item in self.tree['nodes']:
            if item['type']=="HasAValue":
                self.nodes[item['featureID']] = QMHasAValue(self, item)

            elif item['type']=="InAList":
                self.nodes[item['featureID']] = QMInAList(self, item)

            elif item['type']=="List":
                self.nodes[item['featureID']] = QMList(self, item)
                
            elif item['type']=="TwoDates":
                self.nodes[item['featureID']] = QMTwoDates(self, item)


            # elif item['type']=="Boolean":
            #     self.nodes[item['featureID']] = QMBoolean(self, item)

            # elif item['type']=="Less":
            #     self.nodes[item['featureID']] = QMLess(self, item)

            # elif item['type']=="Larger":
            #     self.nodes[item['featureID']] = QMLarger(self, item)

            # elif item['type']=="inRange":
            #     self.nodes[item['featureID']] = QMinRange(self, item)

            elif item['type'] == "end" :
                self.nodes[item['featureID']] = QMEndNode(self, item)

        
    def addNode(self, node):
        pass
    
    def removeNode(self, node):
        pass
    
    def getNode(self, id):
        try:
            return self.nodes[id]
        except:
            print('Warning: no such node with id ' , id)
            return None
        
    def evaluate(self, startId, patientData):
        return self.getNode(startId).evaluate(patientData)

         
           
    def get_next(self, next_node, dt_json):
        '''search for the next item'''
        print('Next node is : ', next_node)
        for node_dict in dt_json:
            if node_dict['featureID']==next_node:
                return node_dict



In [105]:
# with open('Data/data.json', 'r') as f:
#     dataset = json.load(f)
# results = [myTree.evaluate("1", patient) for patient in dataset]
# ----------------------------------------------------------------
# ----------------------------------------------------------------

myTree = QMTree()
myTree.readTreeFile("Data/QMs/QM2")

# Working on Signednoteprostate
# myFile = 'signednote1'
myFile = 'Rishabhsignednote'

with open(f'Data/PatientsData/v2/{myFile}.json') as file:
    OneItem = json.load(file)
    
results= myTree.evaluate("1", OneItem)

print("Pass patients: ", results.count("pass"))
print("Fail patients: ", results.count("fail"))
print("Exclude patients: ", results.count("exclude"))

# print(myTree.evaluate("1", dataset[0]) )




Evaluating the following expression:	 presentOncologic.radiotherapy
presentOncologic
radiotherapy
QMInAList: True

Evaluating the following expression:	 boneScan.date.$date.$numberLong
boneScan
date
$date
$numberLong

Evaluating the following expression:	 boneScan.date.$date.$numberLong
boneScan
date
$date
$numberLong
QMTwoDates1: 2021-11-17 19:00:00
QMTwoDates2: 2021-11-17 19:00:00
Pass patients:  1
Fail patients:  0
Exclude patients:  0
